# Fluid Structure Interaction with SpatialPy

## Definition of the model

### Imports and definitions

In [ ]:
import os
import sys
sys.path.insert(1, "../..")
import math
import numpy as np
import matplotlib.pyplot as plt

import spatialpy

In [ ]:
class Density(spatialpy.BoundaryCondition):
    def __init__(self, rhoF, rhoB):
        self.rhoF = rhoF
        self.rhoB = rhoB
        
    def expression(self):
        bcstr = "if(me->type == 1 || me->type == 3){"
        bcstr += f"me->rho = {self.rhoF};"
        bcstr += "}"
        bcstr += "if(me->type == 2){"
        bcstr += f"me->rho = {self.rhoB};"
        bcstr += "}"
        return bcstr

In [ ]:
class Teleport(spatialpy.BoundaryCondition):
    def __init__(self, nu, v0):
        self.nu = nu
        self.v0 = v0
        
    def expression(self):
        bcstr = "if(me->x[0] > system->xhi || me->x[0] < system->xlo){"
        bcstr += "me->x[0] = system->xlo + 1e-6;"
        bcstr += f"me->v[0] = {self.v0};"
        bcstr += f"me->nu = {self.nu};"
        bcstr += "}"
        bcstr += "if(me->x[1] > system->yhi){"
        bcstr += "me->v[1]= 0.0;"
        bcstr += "me->x[1]= 99e-6;"
        bcstr += "}"
        bcstr += "if(me->x[1] < system->ylo){"
        bcstr += "me->v[1]= 0.0;"
        bcstr += "me->x[1]= 1e-6;"
        bcstr += "}"
        bcstr += "me->x[2] = 0;"
        return bcstr

### Model

In [ ]:
class FluidStructureInteraction(spatialpy.Model):
    FLUID = 1
    WALLS = 2
    BEAM = 3
    
    def __init__(self, model_name="Fluid Structure Interaction", Ra=1e4):
        spatialpy.Model.__init__(self, model_name)
        
        # System Constants
        Lxint = 300e-6
        Lyint = 100e-6
        Lbzint = -50e-6
        Nwall = 3
        rhof0 = 1000              # Density of fluid
        rhob0 = 7850              # Density of beam
        nu = 1e-3                 # Inlet viscosity
        v0 = 0.0333               # Inlet velocity
        Pratio = 0.33             # Poisson's Ratio
        E = 2e5                   # Young's modulus
        G = E/(2*(1+Pratio))      # Shear Modulus
        K = E/(3*(1-2*Pratio))    # Bulk Modulus
        cb0 = math.sqrt(K/Pratio) # Artificial speed of sound of beam
        cf0 = 10*v0               # Artificial speed of sound of fluid
        
        # Discretization
        Npx = 60
        
        # Compute Domain Bounds
        deltaf = Lyint/Npx
        deltab = 0.6*deltaf
        Wthick = Nwall*deltaf
        Lx = Lxint-Lbzint
        Ly = Lyint+2*Wthick
        yminint = 0.0
        ymaxint = Lyint
        ymin = yminb = -Wthick
        ymax = Lyint+Wthick
        xminb = 100e-6
        xmaxb = 105e-6
        ymaxb = 50e-6
        cy = Lyint/2
        
        # Compute Volume and Mass per Particle
        vol = Lx*Ly
        Wvol = 2*Wthick*Lx
        Fvol = (xmaxb-xminb)*(ymaxb-yminb)
        Bvol = vol-Wvol-Fvol
        mPFP = (Fvol*rhof0)/(Npx*((0-(-50e-6))/deltaf))
        mPBP = (Bvol*rhob0)/(((xmaxb-xminb)/deltab)*((ymaxb-ymin+deltab)/deltab))
        
        # Domain
        eps = 1e-12
        h = 3*deltaf
        
        domain = spatialpy.Domain(
            0, xlim=(Lbzint, Lxint), ylim=(ymin, ymax), zlim=(0,0), gravity=[0, -1, 0]
        )
        # Fluid and Wall Regions
        for x in np.arange(Lbzint, Lxint, deltaf):
            # Fluid Region
            for y in np.arange(yminint, ymaxint, deltaf):
                if x < 0:
                    domain.add_point(
                        [x, y, 0], type=self.FLUID, vol=Fvol, mass=mPFP, nu=nu, fixed=False
                    )
            # Top Wall Region
            for y in np.arange(ymin, yminint, deltaf):
                domain.add_point(
                    [x, y, 0], type=self.WALLS, vol=Wvol, mass=mPFP, nu=nu, fixed=True
                )
            # Bottom Wall Region
            for y in np.arange(ymaxint, ymax, deltaf):
                domain.add_point(
                    [x, y, 0], type=self.WALLS, vol=Wvol, mass=mPFP, nu=nu, fixed=True
                )
        # Beam Region
        for x in np.arange(xminb, xmaxb, deltab):
            for y in np.arange(ymin, ymaxb, deltab):
                domain.add_point(
                    [x, y, 0], type=self.BEAM, vol=Bvol, mass=mPBP, nu=nu, fixed=False
                )
        domain.dimensions = 2
        self.add_domain(domain)
        
        # Static Domain
        self.staticDomain = False
                
        # Boundary Conditions
        self.add_boundary_condition(Density(rhof0, rhob0))
        self.add_boundary_condition(Teleport(nu, v0))
                
        # Timespan
        dt = 1e-8
        nt = 1000000000
        freq_results = 10000
        self.timespan(np.arange(0, nt*dt+dt, freq_results*dt), timestep_size=dt)

In [ ]:
model = FluidStructureInteraction()

In [ ]:
model.domain.plot_types()

## Running model and processing the results

In [ ]:
from spatialpy import Solver
solver = Solver(model=model, debug_level=0)
%time solver.compile()
print(solver.build_dir)

In [ ]:
%time results = solver.run()

In [ ]:
results.plot_property("type",t_ndx=0)

In [ ]:
results.plot_property("type",t_ndx=1)

In [ ]:
#results.plot_property("type", animated=True)

In [ ]:
points, properties = results.read_step(2)
for i, point in enumerate(points):
    if properties['type'][i] == 1:
        print(point)

In [ ]:
print(model.domain.xlim)

In [ ]:
print(model.domain.ylim)